# Spectrograms - CNN Test

In [4]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sqlalchemy
from sqlalchemy import create_engine, inspect

import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

%run functions.ipynb

In [5]:
# Time the run
start_time = time.time()

## Import datasets

In [6]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'demographic',
 'diagnosis',
 'gval',
 'habits',
 'rval',
 'spectrogram']

In [7]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [14]:
# Input shape
width_px = 225
height_px = 166
input_shape = (height_px, width_px)

In [16]:
# Rename the dataframes
df_r = dataframes['rval_df']
df_g = dataframes['gval_df']
df_b = dataframes['bval_df']
df_a = dataframes['aval_df']

# Reshape each RGBA dataframe to original dimensions
data_r = np.array([df_r[col].values.reshape(input_shape) for col in df_r.columns])
data_g = np.array([df_g[col].values.reshape(input_shape) for col in df_g.columns])
data_b = np.array([df_b[col].values.reshape(input_shape) for col in df_b.columns])
data_a = np.array([df_a[col].values.reshape(input_shape) for col in df_a.columns])

rgba_combined = np.stack([
    data_r,
    data_g,
    data_b,
    data_a],
    axis = -1
)

rgba_combined[:3]

array([[[[ 47,  17,  99, 255],
         [ 47,  17,  99, 255],
         [ 43,  16,  93, 255],
         ...,
         [ 35,  11,  70, 255],
         [ 47,  16,  90, 255],
         [ 49,  17,  93, 255]],

        [[ 45,  17,  97, 255],
         [ 45,  17,  98, 255],
         [ 41,  16,  90, 255],
         ...,
         [ 49,  15,  93, 255],
         [ 59,  16, 108, 255],
         [ 60,  15, 111, 255]],

        [[ 46,  17,  98, 255],
         [ 46,  17,  99, 255],
         [ 43,  16,  92, 255],
         ...,
         [ 29,  13,  68, 255],
         [ 51,  16,  99, 255],
         [ 54,  16, 104, 255]],

        ...,

        [[215,  69, 107, 255],
         [215,  69, 107, 255],
         [211,  67, 109, 255],
         ...,
         [ 76,  18, 120, 255],
         [184,  56, 115, 255],
         [201,  62, 114, 255]],

        [[235,  90,  96, 255],
         [235,  90,  96, 255],
         [231,  87,  98, 255],
         ...,
         [ 91,  20, 125, 255],
         [206,  70, 105, 255],
         